In [2]:
import h5py
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split
import tensorflow as tf

/home/anrunw/anaconda3/envs/tfgpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/anrunw/anaconda3/envs/tfgpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/anrunw/anaconda3/envs/tfgpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/anrunw/anaconda3/envs/tfgpu/lib/python3.6/s

### Go Through File and Create Dataframes

In [15]:
f = h5py.File('processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth_0.z','r')
treeArray = f['t_allpar_new'][()]

features = ['j1_ptrel','j1_etarot','j1_phirot','j_index']
featuresUsed = ['j1_ptrel', 'j1_etarot', 'j1_phirot', 'j1_erel', 'j1_deltaR', 'j1_pdgid','j_index']
pdgid = ['j_index', 'j1_pdgid']
glob = ['j_mass_mmdt', 'j_mass' ,'j_index']
labels = ['j_g', 'j_q', 'j_w', 'j_z', 'j_t', 'j_index']

features_df = pd.DataFrame(treeArray,columns=list(set(featuresUsed)))


In [16]:
features_df

,j1_phirot,j1_ptrel,j1_pdgid,j1_erel,j_index,j1_etarot,j1_deltaR
0,0.000000,0.386118,130.0,0.382772,101333,0.000000,0.011056
1,0.007236,0.107734,211.0,0.105703,101333,-0.009859,0.023104
2,-0.002759,0.094731,321.0,0.094339,101333,0.004508,0.006306
3,-0.014909,0.042086,211.0,0.042913,101333,0.028567,0.022473
4,-0.003745,0.039412,-211.0,0.038547,101333,-0.018020,0.025492
...,...,...,...,...,...,...,...
5131608,-0.315306,0.000350,111.0,0.000344,300123134,-0.300495,0.380018
5131609,-0.349366,0.000258,111.0,0.000256,300123134,0.258936,0.294476
5131610,-0.270430,0.000219,111.0,0.000215,300123134,0.269149,0.255231
5131611,-0.341398,0.000116,111.0,0.000115,300123134,-0.213081,0.319973


In [17]:
features_df['j_index'].drop_duplicates()

0             101333
51         100073064
101        300177633
142        100188310
161        200044159
             ...    
5131388    300173707
5131417    200033311
5131447    400153985
5131486    300037743
5131530    300123134
Name: j_index, Length: 98769, dtype: int32

### Group Features in Dataframe by Jet Index

In [18]:
index_df = features_df.groupby(['j_index'])

In [19]:
index_df.first()

,j1_phirot,j1_ptrel,j1_pdgid,j1_erel,j1_etarot,j1_deltaR
j_index,,,,,,
1,0.0,0.526719,130.0,0.525633,0.0,0.024813
3,0.0,0.689462,111.0,0.699932,0.0,0.057165
7,0.0,0.390835,13.0,0.388123,0.0,0.079966
21,0.0,0.133965,-211.0,0.130639,0.0,0.044376
23,0.0,0.130030,321.0,0.129065,-0.0,0.061479
...,...,...,...,...,...,...
400199939,0.0,0.079498,-11.0,0.068866,0.0,0.226768
400199945,0.0,0.162039,-211.0,0.158689,0.0,0.061562
400199962,0.0,0.067352,-211.0,0.063112,0.0,0.262295


In [20]:
index_df.last()

,j1_phirot,j1_ptrel,j1_pdgid,j1_erel,j1_etarot,j1_deltaR
j_index,,,,,,
1,-0.190216,4.929822e-15,111.0,4.914950e-15,0.030484,0.167938
3,0.077529,1.743528e-04,11.0,1.716514e-04,-0.180854,0.216846
7,-0.431896,2.259763e-15,111.0,2.267890e-15,-0.113552,0.371562
21,0.129893,1.921491e-14,111.0,1.758091e-14,0.029871,0.176887
23,-0.016355,1.727544e-04,11.0,1.724079e-04,0.203801,0.210257
...,...,...,...,...,...,...
400199939,-0.174553,1.899589e-04,111.0,1.702099e-04,0.270294,0.285076
400199945,0.142462,1.980837e-13,130.0,1.999639e-13,-0.100336,0.211512
400199962,-0.508617,1.629955e-04,111.0,1.746291e-04,-0.290382,0.318107


In [21]:
max_particles = 20

### Iterate through dataframe to create a numpy array with desired dimensions


In [30]:
data = np.zeros([98769,max_particles,6])
p = 0
for name, group in index_df:
    x = group.to_numpy()
    x = np.delete(x,4,1)
    g = 0
    for l in x:
        if g == 20: 
            break
        for n in range(6):
            data[p,g,n] = l[n]
        g = g+1

    p = p+1
    

In [31]:
data

array([[[ 0.00000000e+00,  5.26718855e-01,  1.30000000e+02,
          5.25633097e-01,  0.00000000e+00,  2.48127822e-02],
        [ 4.55306284e-03,  8.16964805e-02, -3.21000000e+02,
          8.15227404e-02, -1.00144953e-03,  2.94277072e-02],
        [-4.35186643e-03,  7.24203736e-02, -2.11000000e+02,
          7.22680390e-02,  6.44924934e-04,  2.04328783e-02],
        ...,
        [-2.80741543e-01,  5.33214770e-03,  1.11000000e+02,
          5.23904618e-03,  5.28131891e-03,  2.56084561e-01],
        [ 9.37831998e-02,  5.15896268e-03,  2.11000000e+02,
          5.13469940e-03, -2.41304729e-02,  1.21284731e-01],
        [-2.45866120e-01,  3.94729385e-03, -2.11000000e+02,
          3.89839825e-03,  1.28893116e-02,  2.21420020e-01]],

       [[ 0.00000000e+00,  6.89462423e-01,  1.11000000e+02,
          6.99931800e-01,  0.00000000e+00,  5.71646057e-02],
        [-2.49775752e-01,  1.68542340e-01,  1.30000000e+02,
          1.59876242e-01,  8.34689102e-14,  1.94951132e-01],
        [-6.19659

### Create and group labels dataframe by index

In [32]:
labels_df = pd.DataFrame(treeArray,columns=list(set(labels)))


In [33]:
labels_df

,j_g,j_q,j_w,j_index,j_t,j_z
0,1,0,0,101333,0,0
1,1,0,0,101333,0,0
2,1,0,0,101333,0,0
3,1,0,0,101333,0,0
4,1,0,0,101333,0,0
...,...,...,...,...,...,...
5131608,0,0,0,300123134,0,1
5131609,0,0,0,300123134,0,1
5131610,0,0,0,300123134,0,1
5131611,0,0,0,300123134,0,1


In [34]:
labels_df = labels_df.groupby('j_index')

### Iterate through Labels Dataframe to return Numpy Array of labels

In [35]:
labels_df.first()

,j_g,j_q,j_w,j_t,j_z
j_index,,,,,
1,1,0,0,0,0
3,1,0,0,0,0
7,1,0,0,0,0
21,1,0,0,0,0
23,1,0,0,0,0
...,...,...,...,...,...
400199939,0,0,0,1,0
400199945,0,0,0,1,0
400199962,0,0,0,1,0


In [41]:
labels = np.zeros([98769,5])
o = 0
for name,  group in labels_df:
    x = group.to_numpy()
    x = np.delete(x,3,1)
    for z in range(5):
        labels[o,z] = x[0][z]

    o = o+1

In [50]:
np.amax(x)

1

### Group global features by index and iterate to return a numpy array in desired shape

In [51]:
globdf = pd.DataFrame(treeArray,columns = glob)

In [52]:
globdf = globdf.groupby('j_index')

In [53]:
glob = np.zeros([98769, 2])
o = 0
for name, group in globdf:
    x = group.to_numpy()
    x = np.delete(x,2,1)
    for m in range(2):
        glob[o,m] = x[0,m]
    o = o+1

In [54]:
glob


array([[ 90.77446747,  93.24835968],
       [148.98461914, 150.47558594],
       [241.29042053, 241.29042053],
       ...,
       [182.7910614 , 182.7910614 ],
       [145.66259766, 145.66259766],
       [166.74099731, 166.74099731]])

In [55]:
labels.shape

(98769, 5)

### Split into train and validation/test sets

In [56]:
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.50, random_state=42)


In [57]:
trash_train, trash_test, glob_train, glob_test = train_test_split(data, glob, test_size=0.50, random_state=42)

In [58]:
labels_train.shape

(49384, 5)

In [59]:
data_train.shape

(49384, 20, 6)

In [60]:
glob_train.shape

(49384, 2)

In [61]:
data_test.shape

(49385, 20, 6)

### Create File

In [62]:
f = h5py.File('trainABC6feat5tag.h5','w')

In [63]:
f.create_dataset('data', data = data_train)

<HDF5 dataset "data": shape (49384, 20, 6), type "<f8">

In [64]:
f.create_dataset('pid', data = labels_train)

<HDF5 dataset "pid": shape (49384, 5), type "<f8">

In [65]:
f.create_dataset('global', data = glob_train)

<HDF5 dataset "global": shape (49384, 2), type "<f8">

In [66]:
f.keys()


<KeysViewHDF5 ['data', 'global', 'pid']>

In [67]:
f.close()

### Split rest of data into the validation and test sets

In [68]:
data_val, datatest, labels_val, labels_test = train_test_split(data_test, labels_test, test_size=0.50, random_state=42)


In [69]:
trash_val, trash_test, glob_val, glob_test = train_test_split(data_test, glob_test, test_size = 0.50, random_state=42)

### Create Files

In [70]:
f = h5py.File('testABC6feat5tag.h5', 'w')

In [71]:
f.create_dataset('data', data = datatest)


<HDF5 dataset "data": shape (24693, 20, 6), type "<f8">

In [72]:
f.create_dataset('pid', data = labels_test)


<HDF5 dataset "pid": shape (24693, 5), type "<f8">

In [73]:
f.create_dataset('global', data = glob_test)

<HDF5 dataset "global": shape (24693, 2), type "<f8">

In [74]:
f.close()

In [75]:
f = h5py.File('valABC6feat5tag.h5', 'w')

In [76]:
f.create_dataset('data', data = data_val)


<HDF5 dataset "data": shape (24692, 20, 6), type "<f8">

In [77]:
f.create_dataset('pid', data = labels_val)


<HDF5 dataset "pid": shape (24692, 5), type "<f8">

In [78]:
f.create_dataset('global', data = glob_val)

<HDF5 dataset "global": shape (24692, 2), type "<f8">

In [79]:
f.close()

In [80]:
labels

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.]])

In [111]:
f = h5py.File('trainABC6feat5tag.h5','r')

In [113]:
f['pid'][:][:,0].shape

(49384,)